In [40]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,auc
from sklearn.model_selection import GridSearchCV


from lightgbm import LGBMClassifier

In [31]:
random.seed(42)
np.random.seed(42)

In [78]:
train = pd.read_csv("../data/train.csv", index_col="Id")
test = pd.read_csv("../data/test.csv", index_col="Id")
train = train.drop(columns="BUTTER")
test = test.drop(columns="BUTTER")

In [79]:
train

,B_OWNPV_CHI2,B_IPCHI2_OWNPV,B_FDCHI2_OWNPV,B_DIRA_OWNPV,B_PT,Kst_892_0_IP_OWNPV,Kst_892_0_cosThetaH,Kplus_IP_OWNPV,Kplus_P,piminus_IP_OWNPV,piminus_P,gamma_PT,piminus_ETA,Kplus_ETA,signal
Id,,,,,,,,,,,,,,,
0,28.878847,2.662533,2924.690991,0.999997,19085.568945,0.569198,-0.575502,0.581565,66850.893711,0.637969,14298.486178,7940.694301,2.628526,2.680116,1.0
1,34.233566,0.092746,346.948714,0.999997,6631.244546,0.248707,-0.615941,0.277898,39274.475071,0.148815,11553.163934,3904.681337,3.292504,3.085754,1.0
2,36.113632,2.442423,238.553023,0.999986,7740.918989,0.222347,0.249383,0.216576,27757.153899,0.249840,24081.196003,4738.891687,3.433676,3.121906,1.0
3,14.286133,6.337556,227.375132,0.999806,6740.281614,0.347316,0.591884,0.306927,10593.207077,0.400748,11343.521945,3308.943750,2.291867,2.200712,0.0
4,60.474274,7.632751,106.730650,0.999905,5556.388794,0.204273,0.655850,0.196600,11801.249543,0.223101,25940.693317,4026.326871,3.290073,3.281829,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212657,33.269855,0.422886,528.518223,0.999999,6547.421388,0.324063,-0.397384,0.286336,50331.007785,0.539486,8898.424053,3657.986285,3.077444,3.243626,1.0
212658,30.405409,6.838541,245.848798,0.999909,6634.286140,0.286608,0.522046,0.194381,32017.565715,0.446696,22491.555816,4975.574892,3.510440,3.603324,0.0
212659,8.028983,2.256855,377.706643,0.999878,5160.700365,0.626228,-0.176330,0.598550,16262.726940,0.705622,5497.144994,4276.250211,2.379503,2.772514,1.0


In [80]:
X_full = train.drop(columns="signal")
X_test = test.copy()
y_full = train.signal

X_train, X_valid, y_train, y_valid = train_test_split(
    X_full, 
    y_full, 
    stratify=train.signal
)

In [35]:
lgb = LGBMClassifier()
lgb.fit(X_train, y_train)

LGBMClassifier()

In [38]:
pred_valid = lgb.predict_proba(X_valid)[:, 1]
pred_train = lgb.predict_proba(X_train)[:, 1]

In [37]:
roc_auc_score(y_valid, pred_valid)

0.8491354170064807

In [39]:
roc_auc_score(y_train, pred_train)

0.8656079707009876

In [87]:
lgb_grid = GridSearchCV(lgb, param_grid={'n_estimators': [100, 200, 500]}, cv=3, scoring='roc_auc')

In [88]:
lgb_grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=LGBMClassifier(),
             param_grid={'n_estimators': [100, 200, 500]}, scoring='roc_auc')

In [89]:
lgb_grid.best_score_

0.8618902333445336

In [90]:
lgb_grid.best_params_

{'n_estimators': 500}

In [91]:
lgb_best = LGBMClassifier(n_estimators=500)
lgb_best.fit(X_train, y_train)

LGBMClassifier(n_estimators=500)

In [92]:
pred_valid = lgb_best.predict_proba(X_valid)[:, 1]
pred_train = lgb_best.predict_proba(X_train)[:, 1]

In [93]:
roc_auc_score(y_valid, pred_valid)

# 1000: valid 866, cv 8649
# 500: valid 868, cv 861

0.8689227046310779

In [94]:
roc_auc_score(y_train, pred_train)

0.910944177499504

In [75]:
lgb_best.fit(X_full, y_full)

LGBMClassifier(n_estimators=1000)

In [77]:
X_test

,B_OWNPV_CHI2,B_IPCHI2_OWNPV,B_FDCHI2_OWNPV,B_DIRA_OWNPV,B_PT,Kst_892_0_IP_OWNPV,Kst_892_0_cosThetaH,Kplus_IP_OWNPV,Kplus_P,piminus_IP_OWNPV,piminus_P,gamma_PT,piminus_ETA,Kplus_ETA,Predicted
Id,,,,,,,,,,,,,,,
0,20.173445,3.846368,120.871099,0.999868,6160.068877,0.283769,0.784221,0.221518,14938.796613,0.334843,18885.484755,5739.899165,3.395456,3.520709,0.023993
1,22.472050,3.463131,23290.089549,1.000000,4421.210474,2.289943,-0.755909,2.675985,27997.248030,1.512916,8658.765482,3511.829361,3.498786,3.427056,0.289276
2,13.758772,6.239888,2826.232845,0.999946,6271.957675,0.860737,0.321553,0.964498,8383.679923,0.846215,10310.459345,3211.637797,2.401201,2.224787,0.126295
3,39.320282,0.146056,992.465454,0.999998,7440.960902,0.485886,0.371864,0.457798,23595.091189,0.576270,13975.989410,5466.588206,3.203331,3.491705,0.472685
4,35.042408,3.912886,830.923177,0.999830,6120.895612,0.620750,0.325027,0.590861,6875.570461,0.721326,5065.980778,5250.449109,2.518374,2.190288,0.520069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141771,30.533255,1.699120,247.567972,0.999888,8367.474105,0.221315,0.478428,0.300806,10682.845309,0.187368,25552.669086,6022.967681,3.105870,3.033990,0.011321
141772,45.894135,0.304761,98.141535,0.999989,8777.501344,0.265711,-0.512062,0.241356,69159.089014,0.393407,10720.653365,5716.381079,3.508286,3.815329,0.398867
141773,43.790756,2.437984,320.666560,0.999974,6736.548775,0.233608,0.073552,0.239403,37153.397051,0.238732,25575.359107,4168.692210,3.629445,3.312150,0.516379


In [81]:
test_predictions = lgb_best.predict_proba(X_test)[:, 1]

In [82]:
test['Predicted'] = test_predictions

In [83]:
test[['Predicted']].to_csv('../submissions/first_lgbm.csv')